In [58]:
import json
import os
import ast
import pandas as pd
rest_reviews=pd.read_csv('restaurant_reviews_newcols.csv',nrows=10000)

In [ ]:
# emphasis_text=[x for x in rest_reviews['text'] if isinstance(x,str)]
# emphasis_count=[x.count('!') for x in rest_reviews['text'] if isinstance(x,str)]
# temp=pd.DataFrame()
# temp['text']=emphasis_text
# temp['count']=emphasis_count

In [ ]:
# check=temp[temp['count']>50]['text']

In [ ]:
def category_filter(df):
    import re
    global search_category
    searched=search_category.split(' ')
    
    if isinstance(df['text'],str):
        temp=df['text'].split(', ')
        checksum=sum([searched.count(x) for x in temp])
        return checksum
    else:
        return 0

In [ ]:
search_category=input('What are you searching for today?')
rest_reviews['flag']=rest_reviews.apply(category_filter,axis=1)
[print(i) for i in rest_reviews[rest_reviews['flag']>0]]

In [2]:
#Trying to use LDA to model topics for reviews
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint

In [59]:
documents=rest_reviews['text']
stoplist = list(STOPWORDS)
texts = [[word for word in document.lower().split() if (word not in stoplist) & (word.isalnum())]
         for document in documents if isinstance(document,str)]
texts

[['pizza',
  'best',
  'prefer',
  'flamingo',
  'fort',
  'chef',
  'better',
  'ny',
  'style',
  'pizzeria',
  'cosmo',
  'priced',
  'quality',
  'lack',
  'personality',
  'better',
  'pick',
  'youre',
  'going',
  'italian',
  'family',
  'home',
  'people',
  'actually',
  'care',
  'like',
  'dont',
  'pizzeria',
  'dont',
  'care'],
 ['love',
  'fiance',
  'atleast',
  'portions',
  'food',
  'love',
  'carne',
  'great',
  'lunch',
  'leticia',
  'super',
  'nice',
  'cares',
  'think',
  'try',
  'cheese',
  'enchiladas',
  'sauce',
  'different'],
 ['dry',
  'corn',
  'rib',
  'tips',
  'fat',
  'mushy',
  'want',
  'bbq',
  'neighborhood',
  'john',
  'mulls',
  'roadkill',
  'trust'],
 ['place',
  'favorite',
  'thai',
  'place',
  'alllll',
  'started',
  'known',
  'got',
  'service',
  'started',
  'suck',
  'portion',
  'sizes',
  'got',
  'cut',
  'huge',
  'problem',
  'paying',
  'way',
  'time',
  'went',
  'pork',
  'pad',
  'se',
  'ew',
  'tasted',
  'finished

In [60]:
#In the above code, we tokenize all of the reviews and remove stop words
#remove words that appear only once 
#frequency of the token across ALL review

#counting the frequency of token across ALL reviews
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
#removing tokens that occur only once across all reviews
texts = [[token for token in text if frequency[token] > 1] for text in texts] #if frequence(token)>1

In [61]:
dictionary = corpora.Dictionary(texts) #word id-word pairs
corpus = [dictionary.doc2bow(text) for text in texts] #word id-frequence of word pairs

In [29]:
import collections as c
emptylist=[]
[emptylist.extend(x) for x in texts]
len(emptylist)
count=c.Counter(emptylist)
count['great']

3619

In [31]:
#Set parameters
num_topics = 10
passes = 50
lda = LdaModel(corpus,
              id2word=dictionary,
              num_topics=num_topics,
              passes=passes)

In [32]:
lda.print_topics()

[(0,
  '0.028*"place" + 0.027*"food" + 0.022*"service" + 0.015*"like" + 0.013*"good" + 0.010*"sushi" + 0.010*"time" + 0.009*"customer" + 0.008*"better" + 0.008*"know"'),
 (1,
  '0.135*"pizza" + 0.017*"crust" + 0.014*"wine" + 0.011*"delivery" + 0.007*"chicago" + 0.006*"bottle" + 0.006*"latte" + 0.005*"best" + 0.005*"falafel" + 0.005*"mix"'),
 (2,
  '0.020*"food" + 0.016*"came" + 0.015*"asked" + 0.015*"got" + 0.015*"ordered" + 0.015*"said" + 0.014*"minutes" + 0.014*"time" + 0.013*"took" + 0.013*"order"'),
 (3,
  '0.013*"ordered" + 0.012*"chicken" + 0.012*"good" + 0.011*"salad" + 0.010*"sauce" + 0.009*"fried" + 0.009*"like" + 0.009*"cheese" + 0.007*"cooked" + 0.007*"steak"'),
 (4,
  '0.027*"like" + 0.015*"chicken" + 0.012*"food" + 0.011*"place" + 0.009*"order" + 0.009*"got" + 0.009*"good" + 0.009*"ordered" + 0.008*"time" + 0.007*"eat"'),
 (5,
  '0.013*"free" + 0.008*"gluten" + 0.008*"donuts" + 0.007*"actually" + 0.007*"dog" + 0.007*"room" + 0.006*"door" + 0.006*"opened" + 0.005*"hotel" + 

In [ ]:
#after multiple attempts, trying LSI model, instead of LDA

from gensim.similarities.docsim import Similarity
from gensim import corpora, models, similarities
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=15)

doc=input('what are you searching for today?')
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
index = similarities.MatrixSimilarity(lsi[corpus])
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims[0][0]

In [73]:

[print(documents[sims[i][0]],'\n') for i in range(5)]

Excellent Seafood. Top notch staff.  Our server Cindy was awesome!  Just checked in and received a free glass of wine! 

We have been waiting to try this new spot...first and foremost, nice atmosphere for a Saturday afternoon...our server was "okay", just not really attentive for a non busy place...which is why we  couldn't help notice the table across from us not clean through out our dinning experience.(which every server saw clearly because they looked at it, but refused to fix) Which remained the same until we left, unclean and disgusting, which we still took all in with a smile , but finally complained that our server sucked to another server, which was eager to tell our server. Lets just say our server handled it with " true grace"...needless to say, there is still "one" great server there, which was always getting our server or answering our questions, with a smile and energy....She constantly checked on her tables, with a smile and pure energy,"wishing we sat in her area", beca

[None, None, None, None, None]